## Batch processing with Argo Worfklows

In this notebook we will dive into how you can run batch processing with Argo Workflows and Seldon Core.

Dependencies:

* Seldon core installed as per the docs with an ingress
* Minio running in your cluster to use as local (s3) object storage
* Argo Workfklows installed in cluster (and argo CLI for commands)


## Setup

### Install Seldon Core
Use the notebook to [set-up Seldon Core with Ambassador or Istio Ingress](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html).

Note: If running with KIND you need to make sure do follow [these steps](https://github.com/argoproj/argo/issues/2376#issuecomment-595593237) as workaround to the `/.../docker.sock` known issue.

### Set up Minio in your cluster
Use the notebook to [set-up Minio in your cluster](https://docs.seldon.io/projects/seldon-core/en/latest/examples/minio_setup.html).

### Copy the Minio Secret to namespace

We need to re-use the minio secret for the batch job, so this can be done by just copying the minio secret created in the `minio-system`

The command below just copies the secred with the name "minio" from the minio-system namespace to the default namespace.

In [1]:
!kubectl get secret minio -n minio-system -o json | jq '{apiVersion,data,kind,metadata,type} | .metadata |= {"annotations", "name"}' | kubectl apply -n default -f -

secret/minio created


### Install Argo Workflows
You can follow the instructions from the official [Argo Workflows Documentation](https://github.com/argoproj/argo#quickstart).

You also need to make sure that argo has permissions to create seldon deployments - for this you can just create a default-admin rolebinding as follows:

In [2]:
!kubectl create rolebinding default-admin --clusterrole=admin --serviceaccount=default:default

rolebinding.rbac.authorization.k8s.io/default-admin created


## Create some input for our model

We will create a file that will contain the inputs that will be sent to our model

In [3]:
mkdir -p assets/

In [4]:
with open("assets/input-data.txt", "w") as f:
    for i in range(10000):
        f.write('[[1, 2, 3, 4]]\n')

### Check the contents of the file

In [5]:
!wc -l assets/input-data.txt
!head assets/input-data.txt

10000 assets/input-data.txt
[[1, 2, 3, 4]]
[[1, 2, 3, 4]]
[[1, 2, 3, 4]]
[[1, 2, 3, 4]]
[[1, 2, 3, 4]]
[[1, 2, 3, 4]]
[[1, 2, 3, 4]]
[[1, 2, 3, 4]]
[[1, 2, 3, 4]]
[[1, 2, 3, 4]]


### Upload the file to our minio

In [6]:
!mc mb minio-seldon/data
!mc cp assets/input-data.txt minio-seldon/data/

Bucket created successfully `minio-seldon/data`.
...-data.txt:  146.48 KiB / 146.48 KiB ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 2.14 MiB/s 0s

### Create Argo Workflow

In order to create our argo workflow we have made it simple so you can leverage the power of the helm charts.

Before we dive into the contents of the full helm chart, let's first give it a try with some of the settings.

We will run a batch job that will set up a Seldon Deployment with 10 replicas and 100 batch client workers to send requests.

In [50]:
!helm template seldon-batch-workflow helm-charts/seldon-batch-workflow/ \
    --set workflow.name=seldon-batch-process \
    --set seldonDeployment.name=sklearn \
    --set seldonDeployment.replicas=10 \
    --set seldonDeployment.serverWorkers=1 \
    --set seldonDeployment.serverThreads=10 \
    --set batchWorker.workers=100 \
    --set batchWorker.payloadType=ndarray \
    --set batchWorker.dataType=data \
    | argo submit -

Name:                seldon-batch-process
Namespace:           default
ServiceAccount:      default
Status:              Pending
Created:             Thu Aug 06 08:21:47 +0100 (now)


In [51]:
!argo list

NAME                   STATUS    AGE   DURATION   PRIORITY
seldon-batch-process   Running   2s    2s         0


In [37]:
!argo get seldon-batch-process

Name:                seldon-batch-process
Namespace:           default
ServiceAccount:      default
Status:              Succeeded
Created:             Thu Aug 06 08:03:31 +0100 (1 minute ago)
Started:             Thu Aug 06 08:03:31 +0100 (1 minute ago)
Finished:            Thu Aug 06 08:04:54 +0100 (26 seconds ago)
Duration:            1 minute 23 seconds

STEP                                                             PODNAME                          DURATION  MESSAGE
 ✔ seldon-batch-process (seldon-batch-process)                                                              
 ├---✔ create-seldon-resource (create-seldon-resource-template)  seldon-batch-process-3626514072  2s        
 ├---✔ wait-seldon-resource (wait-seldon-resource-template)      seldon-batch-process-2052519094  28s       
 ├---✔ download-object-store (download-object-store-template)    seldon-batch-process-1257652469  2s        
 ├---✔ process-batch-inputs (process-batch-inputs-template)      seldon-batch-process-2

In [52]:
!argo logs -w seldon-batch-process || argo logs seldon-batch-process # The 2nd command is for argo 2.8+

create-seldon-resource:	time="2020-08-06T07:21:48.400Z" level=info msg="Starting Workflow Executor" version=v2.9.3
create-seldon-resource:	time="2020-08-06T07:21:48.404Z" level=info msg="Creating a docker executor"
create-seldon-resource:	time="2020-08-06T07:21:48.404Z" level=info msg="Executor (version: v2.9.3, build_date: 2020-07-18T19:11:19Z) initialized (pod: default/seldon-batch-process-3626514072) with template:\n{\"name\":\"create-seldon-resource-template\",\"arguments\":{},\"inputs\":{},\"outputs\":{},\"metadata\":{},\"resource\":{\"action\":\"create\",\"manifest\":\"apiVersion: machinelearning.seldon.io/v1\\nkind: SeldonDeployment\\nmetadata:\\n  name: \\\"sklearn\\\"\\n  namespace: default\\n  ownerReferences:\\n  - apiVersion: argoproj.io/v1alpha1\\n    blockOwnerDeletion: true\\n    kind: Workflow\\n    name: \\\"seldon-batch-process\\\"\\n    uid: \\\"401c8bc0-0ff0-4f7b-94ba-347df5c786f9\\\"\\nspec:\\n  name: \\\"sklearn\\\"\\n  predictors:\\n    - componentSpecs:\\n      

## Check output in object store

We can now visualise the output that we obtained in the object store.

First we can check that the file is present:

In [53]:
import json
wf_arr = !argo get seldon-batch-process -o json
wf = json.loads("".join(wf_arr))
WF_ID = wf["metadata"]["uid"]
print(f"Workflow ID is {WF_ID}")

Workflow ID is 401c8bc0-0ff0-4f7b-94ba-347df5c786f9


In [54]:
!mc ls minio-seldon/data/output-data-"$WF_ID".txt

[2020-08-06 08:23:07 BST]  2.7MiB output-data-401c8bc0-0ff0-4f7b-94ba-347df5c786f9.txt


Now we can output the contents of the file created using the `mc head` command.

In [55]:
!mc cp minio-seldon/data/output-data-"$WF_ID".txt assets/output-data.txt
!head assets/output-data.txt

...786f9.txt:  2.75 MiB / 2.75 MiB ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 26.55 MiB/s 0s{"data": {"names": ["t:0", "t:1", "t:2"], "ndarray": [[0.0006985194531162841, 0.003668039039435755, 0.9956334415074478]]}, "meta": {"tags": {"tags": {"batch_id": "95e6e8d0-d7b5-11ea-b00e-ea443eed4c19", "batch_index": 2.0, "batch_instance_id": "95e7df56-d7b5-11ea-b5f2-ea443eed4c19"}}}}
{"data": {"names": ["t:0", "t:1", "t:2"], "ndarray": [[0.0006985194531162841, 0.003668039039435755, 0.9956334415074478]]}, "meta": {"tags": {"tags": {"batch_id": "95e6e8d0-d7b5-11ea-b00e-ea443eed4c19", "batch_index": 0.0, "batch_instance_id": "95e77c3c-d7b5-11ea-b5f2-ea443eed4c19"}}}}
{"data": {"names": ["t:0", "t:1", "t:2"], "ndarray": [[0.0006985194531162841, 0.003668039039435755, 0.9956334415074478]]}, "meta": {"tags": {"tags": {"batch_id": "95e6e8d0-d7b5-11ea-b00e-ea443eed4c19", "batch_index": 1.0, "batch_instance_id": "95e787ae-d7b5-11ea-b5f2-ea443eed4c19"}}}}
{"data": {"names": ["t:0", "t:1", "t:2"], "ndarray": [[0.00069

In [56]:
!argo delete seldon-batch-process

Workflow 'seldon-batch-process' deleted
